In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime

import numpy as np

import warnings
warnings.simplefilter('ignore')

In [2]:
fecha1 = '2023-01-01'
fecha2 = '2023-08-01'

hc = pd.read_csv('cleaned_data.csv')
hc.fecha_out = pd.to_datetime(hc.fecha_out)
hc.fecha_in = pd.to_datetime(hc.fecha_in)

listado = pd.read_csv('listado.csv')

In [3]:
def get_Rotacion(hc, listado, grupos, fecha1, fecha2):
        
    listado = listado.groupby(grupos).count().drop(listado.columns[~listado.columns.isin(grupos)], axis=1)
    
    # ROTACIÓN GENERAL
    entradas = listado.join(hc[(hc.fecha_in>=fecha1) & (hc.fecha_in<fecha2)].groupby(grupos).count()).fillna(0)
    salidas = listado.join(hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2)].groupby(grupos).count()).fillna(0)
    pip = listado.join(hc[(hc.fecha_in<=fecha1) & (hc.fecha_out>=fecha1)].groupby(grupos).count()).fillna(0)
    pfp = listado.join(hc[(hc.fecha_in<fecha2) & (hc.fecha_out>=fecha2)].groupby(grupos).count()).fillna(0)
    rotacion = (salidas/((pip+pfp)/2))[['id_empleado']].rename(columns={'id_empleado':'rotacion'})
    datos_rotacion = [pip.id_empleado.to_frame('pip'), pfp.id_empleado.to_frame('pfp'), salidas.id_empleado.to_frame('salidas'), entradas.id_empleado.to_frame('entradas')]

    # % Salidas del personal que estuvo menos de 3 meses
    pct_3meses = ((listado.join(hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.dias_empresa>=0) & (hc.dias_empresa<=90)].groupby(grupos).count()).fillna(0))/salidas)[['id_empleado']].rename(columns={'id_empleado':'rot_3meses'})

    # % Salidas por motivo de baja
    voluntaria = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.motivo_baja=='Baja voluntaria del trabajador')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('baja_voluntaria')
    despido = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.motivo_baja=='Despido del trabajador')].groupby(grupos).count()).fillna(0)/salidas)[['id_empleado']].id_empleado.to_frame('baja_despido')
    fin_temporal = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.motivo_baja=='Fin de contrato temporal')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('baja_finTemporal')
    otros = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.motivo_baja=='Otros')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('baja_otros')
    tipos_baja = [voluntaria, despido, fin_temporal, otros]

    # % Salidas por horas de contrato
    h1520 = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.horas_contrato=='CONTRATO DE 15 A 20 HORAS')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('h1520')
    h2030 = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.horas_contrato=='CONTRATO DE 20 A 30 HORAS')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('h2030')
    h3040 = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.horas_contrato=='CONTRATO DE 30 A 40 HORAS')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('h3040')
    hDesconocido = listado.join((hc[(hc.fecha_out>=fecha1) & (hc.fecha_out<fecha2) & (hc.horas_contrato=='Desconocido')].groupby(grupos).count()).fillna(0)/salidas).id_empleado.to_frame('hDesconocido')
    horas_contrato = [h1520, h2030, h3040, hDesconocido]

    # Forma que tendrá la tabla final
    rotacion = rotacion.join(pct_3meses).join(tipos_baja).join(horas_contrato).join(datos_rotacion)

    # Rotación por mes
    date1 = '2023-01-01'
    for m in range(1, pd.to_datetime(fecha2).month):
        date2 = (pd.to_datetime(date1) + pd.DateOffset(months=1)).strftime("%Y-%m-%d")
        salidas_mes = listado.join(hc[(hc.fecha_out>=date1) & (hc.fecha_out<date2)].groupby(grupos).count()).fillna(0)
        pip_mes = listado.join(hc[(hc.fecha_in<=date1) & (hc.fecha_out>=date1)].groupby(grupos).count()).fillna(0)
        pfp_mes = listado.join(hc[(hc.fecha_in<date2) & (hc.fecha_out>=date2)].groupby(grupos).count()).fillna(0)
        rotacion_mes = (salidas_mes/((pip_mes+pfp_mes)/2))[['id_empleado']].rename(columns={'id_empleado':f'mes_{m}'})
        rotacion = rotacion.join(rotacion_mes)
        date1 = date2

    data = hc.groupby(grupos).mean()
    rotacion = rotacion.join(data)
    
    return rotacion.iloc[:, :-6]

In [4]:
total = get_Rotacion(hc.copy(), listado, ['id_place','work_place','supervisor','jefe'], fecha1, fecha2)

In [5]:
total.head()

,,,,rotacion,rot_3meses,baja_voluntaria,baja_despido,baja_finTemporal,baja_otros,h1520,h2030,h3040,hDesconocido,pip,pfp,salidas,entradas,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7
id_place,work_place,supervisor,jefe,,,,,,,,,,,,,,,,,,,,,
1,Wesleyland,Todd Duarte,Catherine Walker MD,0.615385,0.166667,0.583333,0.125000,0.208333,0.083333,0.625000,0.083333,0.291667,NaN,41.0,37.0,24.0,20.0,0.155844,0.083333,0.054795,0.109589,0.114286,0.000000,0.138889
2,South Rebecca,John Jones,Kelsey Richardson,0.914286,0.187500,0.500000,NaN,0.375000,0.125000,0.562500,0.312500,0.125000,NaN,18.0,17.0,16.0,15.0,0.387097,0.076923,0.133333,0.058824,0.057143,0.111111,0.171429
3,Aaronfurt,John Jones,Kelsey Richardson,0.875000,0.095238,0.666667,0.095238,0.095238,0.142857,NaN,0.142857,0.857143,NaN,25.0,23.0,21.0,19.0,0.125000,0.325581,0.090909,0.085106,0.136364,0.090909,0.088889
4,Lake Tracyhaven,John Jones,Kelsey Richardson,0.561404,0.312500,0.500000,0.062500,0.187500,0.250000,0.312500,0.187500,0.500000,NaN,29.0,28.0,16.0,15.0,0.218182,0.120000,0.076923,0.160000,0.000000,0.000000,0.036364
6,Brownville,John Jones,Kelsey Richardson,0.687500,0.000000,0.727273,NaN,0.181818,0.090909,0.636364,0.181818,0.181818,NaN,17.0,15.0,11.0,9.0,0.193548,0.142857,0.071429,0.000000,0.068966,0.214286,0.068966
